In [3]:
import tensorflow as tf

if tf.test.is_built_with_cuda():
    print("CUDA is enabled in TensorFlow.")
else:
    print("CUDA is not enabled in TensorFlow.")


CUDA is not enabled in TensorFlow.


In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [5]:
df = pd.read_csv(r'C:\Users\Legend\Downloads\train.csv')
df.head()

,text,label
0,"rt @user olha quem chegouuuuu, nossos queridin...",0
1,veio umas teorias muito loucas na minha cabeça...,1
2,@user @user 😂😂😂😂mais nao tinha falado ontem qu...,0
3,rt @user quer ser filha da puta logo comigo qu...,1
4,vai besta 😂😂😂😂 casquei com a ultima foto,1


In [6]:
# prompt: make a regex that removes all non alphabet characters and numbers and makes them lower case as well and then rename the dataset with the same name

import re

# Remove all non-alphabet characters and numbers, and convert to lowercase
df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x).lower())

# Rename the dataset with the same name
df = df.rename(columns={'text': 'text'})

In [7]:
# prompt: tokensize the text and then add padding of the max length

# Tokenize the text column
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['text'])

# Convert the text to sequences of tokens
sequences = tokenizer.texts_to_sequences(df['text'])

# Pad the sequences to the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=maxlen)


In [8]:
padded_sequences.shape

(16800, 66)

In [ ]:
# prompt: make an embedding layer at the start and then an BiDirectional LSTM layer and then finnaly a linear layer for binary classification 

# Create the model
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(10000,66),  # Embedding layer
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),  # BiDirectional LSTM layer
  tf.keras.layers.Dense(1, activation='sigmoid')  # Linear layer for binary classification
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, df['label'], epochs=15, validation_split=0.2)

In [10]:
# prompt: evaluate the model and test it

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(padded_sequences, df['label'])
print('Test Accuracy:', test_acc)

# Test the model on a new sentence
new_sentence = "I love this movie!"
new_sequence = tokenizer.texts_to_sequences([new_sentence])
padded_new_sequence = pad_sequences(new_sequence, maxlen=maxlen)
prediction = model.predict(padded_new_sequence)
print('Prediction:', prediction)

525/525 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.9795 - loss: 0.0732
Test Accuracy: 0.9261309504508972
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
Prediction: [[1.5041136e-05]]


In [11]:
# Test the model on a new sentence
new_sentence = "user user mais nao tinha falado ontem que n o ia ser patrocinado nada que pudesse violar a moralidade da fam lia brasileira"
new_sequence = tokenizer.texts_to_sequences([new_sentence])
padded_new_sequence = pad_sequences(new_sequence, maxlen=maxlen)
prediction = model.predict(padded_new_sequence)
# Convert the probability prediction to binary label
binary_prediction = 1 if prediction >= 0.5 else 0

# Print the binary prediction
print('Binary Prediction:', binary_prediction)
print('Prediction:', prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Binary Prediction: 0
Prediction: [[5.4570563e-07]]


In [12]:
test_df = pd.read_csv(r"C:\Users\Legend\Downloads\test.csv")
test_df.head()

,id,text
0,0,@user nossa mano te odeio na moral kkkkkkkkkkk...
1,1,@user @user a edição ficou muito boa! kkkkkkkk...
2,2,"@user largada vá, visto que a vaca anda à solta 😂"
3,3,"poxa, eu queria ganhar um boné. alguém me da u..."
4,4,"@user amiga, tudo bem, não precisa se desculpa..."


In [13]:
# Reset the counter
counter = 0

# Create a new column in the test_df to store the predictions
test_df['label'] = np.nan

# Iterate over the test_df and predict each sentence
for index, row in test_df.iterrows():
    # Tokenize the sentence
    new_sequence = tokenizer.texts_to_sequences([row['text']])
    # Pad the sequence to the same length
    padded_new_sequence = pad_sequences(new_sequence, maxlen=maxlen)
    # Predict the sentiment
    prediction = model.predict(padded_new_sequence)
    # Convert the probability prediction to binary label
    binary_prediction = 1 if prediction >= 0.5 else 0
    # Store the prediction in the 'label' column
    test_df.at[index, 'label'] = binary_prediction
    counter += 1
    
    # Print the number of predictions made if counter is divisible by 100
    if counter % 100 == 0:
        print('Number of predictions:', counter)

# Print the updated test_df
print(test_df.head());

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

In [18]:
test_df.head()

,id,label
0,0,0
1,1,0
2,2,1
3,3,0
4,4,1


In [19]:
# prompt: convert label into int and also drop the text column

# Convert label to int
test_df['label'] = test_df['label'].astype(int)

# Drop the text column
test_df = test_df.drop('text', axis=1)

KeyError: "['text'] not found in axis"

In [20]:
test_df.head()

,id,label
0,0,0
1,1,0
2,2,1
3,3,0
4,4,1


In [21]:
test_df.to_csv(r"C:\Users\Legend\Downloads\submission_test.csv", index=False)

In [22]:
test_df.shape

(4200, 2)